#Dewrapping lines

In [ ]:
#get authorization from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/cd_processing/Data/1.working/input/1880/nypl_1880_81_starred.txt"

'/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/nypl_1880_81_starred.txt'


In [1]:
#load data
import glob
file_path = "/content/drive/My Drive/cd_processing/Data/1.working/input/1880/nypl_1880_81_starred.txt"
with open(file_path) as f:
  data = f.readlines()  

##Divide records into different sections based on first letter

Remove special characters from the start of the sentence

In [17]:
# remove special characters from the start of the sentence
for index in range(0, len(data)):
    if (len(data[index]) == 0):
      pass
    elif (data[index][0] in ("'",",","^","!","_","-"," ","’")):
      data[index] = data[index][1:]
    elif (data[index][0] in ("—")):
      data[index] = data[index][1:]
    else:
      pass
    

In [33]:
d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) < 6:
      if line[-3:] == "***":
        # save previous value
        if value:
          if value[0] == "'":
            value = value[1:]
          #print(key)
          d[key] = value
        
        key = line
        value = []

    else:
      value.append(line)
      if key == 'A***':
        d[key] = value



##Combline lines that belong to the same record

Criteria:

1) First letter of next line not equal to the initial letter in this section

2) Special cases in the start of next line: B’klyn, B'way, E., W.

In [34]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0
  
  while i < n:
    if i < n-1:
      if (value[i+1][0] != initial and value[i+1][0] != "*") or (value[i+1][0:2] in ["B’", "E.", "W."]) or (value[i+1][0:2] == ". "):
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value      

In [36]:
#remove \"(see adv....\"\n",
for key,value in d.items():
    n = len(value)
    for i in range(n):
      ls = value[i].split()
      n2 = len(ls)
      for j in range(n2):
        if (("(see" in ls[j]) or ("(See" in ls[j])):
          value[i] = ' '.join(ls[:j])
          break

##Identify business vs person records

In [37]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if (ls[0][0] == "*" ):
        value[i] = value[i][1:]
        d_bp.append([str(index), "b"])
      elif (len(ls[0]) > 1 and ls[0].isupper()):
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

##Save files

In [38]:
file = open("/content/drive/My Drive/cd_processing/Data/1.working/input/1880/nypl_1880_clean_records.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

file2 = open("/content/drive/My Drive/cd_processing/Data/1.working/input/1880/index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()


import the old cleaned file to compare

In [39]:
# import the old cleaned file to compare
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/drive/My Drive/cd_processing/Data/1.working/input/1880/nypl_1880_81_starred_clean_old.txt"
with open(file_path) as f:
  data_clean = f.readlines()


import re
n = len(data_clean)
full_entry = []
for i in range(n):
  ls = data_clean[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      full_entry.append(' '.join(ls[1:]))

Import then new cleaned file

In [40]:
import re
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "/content/drive/My Drive/cd_processing/Data/1.working/input/1880/nypl_1880_clean_records.txt"
with open(file_path) as f:
  new_output = f.readlines()

n = len(new_output)
new_output_check = []
for i in range(n):
  ls = new_output[i].split()
  if ls:
    if bool(re.match("([A-Z]+\.)", ls[0])) != True:
      new_output_check.append(' '.join(ls[1:]))

check how many entries do not match with the old cleaned file

In [41]:
left_new_entries = []

for i in new_output_check:
  if i in full_entry:
    continue
  else:
    left_new_entries.append(i)


print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_new_entries))  # extra records

273114
266624
7285


In [42]:
left_new_entries

['YOUNG & ELLIOTT, seeds, 12 Cortlandt',
 'Young Isaac H. seeds. 12 Cortlandt, h Nyack',
 'York Street Flax Spinning Co. (Limited) 101 Franklin',
 'Warren, B’klyn',
 '(Wust Henry, mason, h r 520 W. 44th',
 'Wustlich Christina, wid. Louis, grocer, 503 W. 29th',
 'Wrought Iron Asphalted Pipe Co. 550 E. 117th',
 'Wright Joseph, umbrellas. 324 B’way',
 'Wright Joseph, variety, 549 W. 26th',
 "Wray L. A. clerk. 170 B'way, h 154 Gates av. B’klyn",
 'Wra.y Margaret A. milliner, 112 W. 17th',
 '15th',
 'Woolworth James, mfr. of handles, 49 Chambers, h Ohio, represented by W. R. McCullough',
 'Worcester Corset Co. 83 Walker',
 'Woodward Francis, police, h 308 Madison',
 'Woodward Frank, machinist, h 350 W. 27th',
 'Woodworth Nason H. carpenter, h 416 W. 41st',
 'Woodworth Robert M. agent, 62 Liberty, h Morris’ Dock',
 'Woodworth William G. clerk, h 84 Perry',
 'Woody Thomas H. waiter, h 158 W. 30th',
 'Woodmansee Luman S. shoes, 131 Duane, h Grove av. c 164th',
 'Woodmansee & Garside, shoes, 13

In [43]:
file = open("/content/drive/My Drive/cd_processing/Data/1.working/input/1880/1880_diff_new_entry.txt", "w")
for record in left_new_entries:
  file.write(record + '\n')
  file.write('\n')

file.close()


check what kind of records present in old file which are not in the newly cleaned file

In [ ]:
left_old_entries = []

for i in full_entry:
  if i in new_output_check:
    continue
  else:
    left_old_entries.append(i)

for i in range(len(left_old_entries)-1):
  if "*" in left_old_entries[i]:
    del left_old_entries[i]


In [60]:

print(len(new_output_check)) # records in new file
print(len(full_entry))      # records in old file
print(len(left_old_entries))  # old records not present in new file

273114
266624
303


In [61]:
left_old_entries

['YOUNG & ELLIOTT,',
 'Young Isaac H.',
 '(Wust Henry, mason, h r 520 W. 44th - Wustlich Christina, wid. Louis, grocer, 503 W. 29th',
 "Wright Joseph, umbrellas. 324 B’way 'Wright Joseph, variety, 549 W. 26th",
 "Wray L. A. clerk. 170 B'way, h 154 Gates av. B’klyn 'Wra.y Margaret A. milliner, 112 W. 17th",
 "Woodward Francis, police, h 308 Madison 'Woodward Frank, machinist, h 350 W. 27th",
 "Woodworth Nason H. carpenter, h 416 W. 41st 'Woodworth Robert M. agent, 62 Liberty, h Mor-",
 'ris’ Dock',
 "Woodworth William G. clerk, h 84 Perry 'Woody Thomas H. waiter, h 158 W. 30th",
 "'Woodhull Caroline W. wid. Wm. S. h 216 E. 45th",
 "Wood H. Duncan, broker, 28 Broad, h 2 E. 15th 'Wood Hamilton, pictures, 139 Eighth, h 17 Lafay-",
 'ette pl',
 'Wood Henrietta L. h 113 Lawrence ,Wood Henry, clerk, h 211 W. 43d',
 "Wolsley Teresa, dressmkr. 202 Lex. av. 'Wolf William, shoes, 2376.5 Fourth av. h 2372",
 "Wolf George, conductor, h 353 Third av. 'Wittmann Elizabeth S. cards, 25 Frankfort, h 73"